In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:



import numpy as np 
import pandas as pd 
import warnings 
warnings.filterwarnings("ignore")
import tensorflow as tf 


print(tf.__version__)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras import Input
from tensorflow.keras.layers import Flatten, Reshape
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,concatenate
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from attention import Attention_Block
# from attention import attention_block
from tensorflow.keras.layers import Attention
from step_attention import AttentionAugmentation2D, _normalize_depth_vars
# from keras.models import load_model

# In[2]:


# from google.colab import drive
# drive.mount('/content/drive')



# In[3]:


from loss import discounted_l1, max_displacement_error,v3_displacement_error
import math
import pickle5 as pickle
import matplotlib.pyplot as plt
from numpy import array
# import horovod.tensorflow.keras as hvd

2.2.0


In [2]:

# hvd.init()

# In[4]:

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"


gpus = tf.config.experimental.list_logical_devices('GPU')
print(gpus)


if gpus:
    
    try:
        
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except:
        print('checking gpu error')
print('checking GPUs')


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config=tf.compat.v1.ConfigProto()
# config.gpu_options.visible_device_list= '0,1'
config.gpu_options.allow_growth=True

config.log_device_placement=True
# config.visible_device_list =2
sess=tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(sess)


# In[5]:


from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import initializers
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split

from load_data import load_data
import os

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
checking gpu error
checking GPUs
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:06:00.0, compute capability: 6.1



In [3]:




# train_X_route = '/home/lab1/repo/lyft/newarray/training_features_vehicle.npy'
# # test_route = home+'/repo/data/X_to_val.npy'

# train_y_route = '/home/lab1/repo/lyft/newarray/training_labels_vehicle.npy'
# test_y_route = home+'/repo/data/y_to_val.npy'
prefix ='/home/lab1/repo/planning/traj_pred/data/'
train_raw, test_raw=None, None
for i in range(1):
    name_tr= prefix+'veh_train_set_8KfB'+'.npy'
    name_te= prefix+'veh_test_set_8KfB'+'.npy'
    tr = np.load(name_tr,allow_pickle=True).astype(float)
    te = np.load(name_te,allow_pickle=True).astype(float)
    if train_raw is None:
        train_raw=tr
    else:
        train_raw= np.vstack((train_raw, tr))
    
    if test_raw is None:
        test_raw=te
    else:
        test_raw= np.vstack((test_raw, te))

print(train_raw.shape, test_raw.shape)




(94440, 11, 4) (2362, 20, 4)


In [4]:
np.random.shuffle(train_raw)
np.random.shuffle(test_raw)
max_n=len(train_raw)
# test_maxn=int(max_n*0.2)
train_set = train_raw[:max_n]
X_train=train_set[:,:10,2:4]
y_train=train_set[:,10,2:4]

# test_set = test_raw[:test_maxn]
test_set = test_raw
X_test=test_set[:,:10,2:4]
y_test=test_set[:,10,2:4]
y = test_set[:,10:,2:4]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, y.shape)

(94440, 10, 2) (94440, 2) (2362, 10, 2) (2362, 2) (2362, 10, 2)


In [5]:
# print(X_position.shape, y.shape, X.shape)
print(X_train.shape, y_train.shape)
print(y_train[0])

(94440, 10, 2) (94440, 2)
[-32.81964111 -50.24951172]


In [6]:
from sklearn.utils import shuffle

n_test=int(len(X_train)*0.9)
X_train, X_val, y_train, y_val= \
X_train[:n_test], X_train[n_test:], y_train[:n_test], y_train[n_test:]
# y=test_set[:n_test,10:,2:4]


print(X_val.shape, y_val.shape)

(9444, 10, 2) (9444, 2)


In [7]:
np.save("train_data/v_xval_8kfb",X_val)
np.save('train_data/v_yval_8kfb',y_val)
np.save("train_data/v_xtrain_8kfb",X_train)
np.save("train_data/v_ytrain_8kfb",y_train)
np.save("train_data/v_xtest_8kfb",X_test)
np.save("train_data/v_ytest_8kfb",y_test)
np.save("train_data/v_y_8kfb",y) # y is the full gt trajectory of xtest, relative coordinate
np.save('train_data/v_test_xall_8kfb',test_set ) # test set, including xtest and y as well as global coordinate

In [8]:
# X_val=np.load("c_xval.npy")
# y_val=np.load('c_yval.npy')
# X_train=np.load("c_xtrain.npy")
# y_train=np.load("c_ytrain.npy")
# X_test=np.load("c_xtest.npy")
# y_test=np.load("c_ytest.npy")
from pathlib import Path
home = str(Path.home())

In [9]:

# In[9]:


filters=20
kernel_size = (3,3)
strides = (1,1)
depth_k =0.2
depth_v =0.2

channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
print("before normalized k, v",depth_k,depth_k,strides)
depth_k, depth_v = _normalize_depth_vars(depth_k, depth_v, filters)
print("after normalized k, v",depth_k,depth_k,strides)


# In[10]:
time_step=10
tf.compat.v1.disable_eager_execution()

rnn = Sequential()
rnn.add(LSTM(128, return_sequences=True, kernel_regularizer='l1', input_shape=(time_step,2)))
rnn.add(LSTM(256, return_sequences=True, kernel_regularizer='l1',input_shape=(time_step,128)))
rnn.add(Reshape((time_step,16,16,1)))
rnn.summary()



cnn = Sequential()

cnn.add(Conv2D(1024, (4, 4),input_shape=(16, 16, 1)))
cnn.add(Conv2D(512, (2, 2)))
cnn.add(Conv2D(256, (4, 4)))
cnn.add(Conv2D(128, (2, 2)))
cnn.add(MaxPooling2D(2,2))
cnn.add(Conv2D(64, (2, 2)))
cnn.add(Conv2D(32, (2, 2)))
cnn.add(Conv2D(16, (2, 2)))

#attention block below
cnn.add(Conv2D(2 * depth_k + depth_k, (1, 1), strides))
cnn.add(AttentionAugmentation2D(depth_k, depth_v, num_heads=4, relative=True))

# cnn.add(Dropout(0.5))
cnn.add(Flatten())

 
cnn.add(Dense(128))
cnn.add(Dense(32))

cnn.summary() 



rnn2=Sequential()
rnn2.add(GRU(256, input_shape=(time_step,32)))


dense = Sequential()
dense.add(Dense(128))
dense.add(Dense(64))
dense.add(Dense(2)) # Model output

main_input = Input(shape=(time_step,2)) 
model = rnn(main_input)
model = TimeDistributed(cnn)(model) 
 # combine timedistributed cnn, attention, and fully connected network with rnn
model = rnn2(model)
model = dense(model) # add dense
final_model = Model(inputs=main_input, outputs=model)
final_model.summary()


before normalized k, v 0.2 0.2 (1, 1)
after normalized k, v 4 4 (1, 1)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 128)           67072     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 256)           394240    
_________________________________________________________________
reshape (Reshape)            (None, 10, 16, 16, 1)     0         
Total params: 461,312
Trainable params: 461,312
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 13, 13, 1024)      17408     
________

In [10]:
# print(home)

In [ ]:


n_batch = 32
n_epoch  = 150


checkpoint_path =home+ "/repo/planning/traj_pred/veh_models/veh_8kfb.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 verbose=1)


 #(None, n_steps, n_features)

sgd = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.5, nesterov=True)
rmsprop =tf.keras.optimizers.RMSprop(learning_rate=0.001)



ftrl = tf.keras.optimizers.Ftrl(learning_rate=0.01)


mtrc = [tf.keras.metrics.MeanAbsoluteError()]
# [tf.keras.metrics.CosineSimilarity(axis=1)]

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=10000,
    decay_rate=0.95)

adagrad = tf.keras.optimizers.Adagrad(learning_rate=lr_schedule)
adadelta = tf.keras.optimizers.Adadelta(learning_rate=lr_schedule)

# to use customized loss, choose from  discounted_l1 or max_displacement_error function as loss parameter 
final_model.compile(loss=v3_displacement_error, optimizer=adadelta, metrics=mtrc)
# final_model.load_weights('/home/lab1/repo/planning/traj_pred/veh_models/veh_139.h5')

history=final_model.fit(X_train, 
                        y_train, epochs=n_epoch, 
                        batch_size=n_batch, 
                        verbose=2, 
                        validation_data=(X_val, y_val),
                        callbacks=[cp_callback])





Train on 84996 samples, validate on 9444 samples
Epoch 1/150

Epoch 00001: val_loss improved from inf to 4.44184, saving model to /home/lab1/repo/planning/traj_pred/veh_models/veh_8kfb.h5
84996/84996 - 343s - loss: 8.9874 - mean_absolute_error: 2.6556 - val_loss: 4.4418 - val_mean_absolute_error: 2.0737
Epoch 2/150

Epoch 00002: val_loss improved from 4.44184 to 3.46065, saving model to /home/lab1/repo/planning/traj_pred/veh_models/veh_8kfb.h5
84996/84996 - 360s - loss: 2.7292 - mean_absolute_error: 0.9843 - val_loss: 3.4607 - val_mean_absolute_error: 1.5475
Epoch 3/150

Epoch 00003: val_loss improved from 3.46065 to 2.42615, saving model to /home/lab1/repo/planning/traj_pred/veh_models/veh_8kfb.h5
84996/84996 - 371s - loss: 2.2991 - mean_absolute_error: 0.8365 - val_loss: 2.4261 - val_mean_absolute_error: 0.9763
Epoch 4/150

Epoch 00004: val_loss did not improve from 2.42615
84996/84996 - 385s - loss: 2.0689 - mean_absolute_error: 0.7591 - val_loss: 2.7702 - val_mean_absolute_error: 1


Epoch 00041: val_loss did not improve from 0.92924
84996/84996 - 345s - loss: 0.9120 - mean_absolute_error: 0.3202 - val_loss: 1.0847 - val_mean_absolute_error: 0.4229
Epoch 42/150

Epoch 00042: val_loss did not improve from 0.92924
84996/84996 - 347s - loss: 0.9025 - mean_absolute_error: 0.3172 - val_loss: 1.3049 - val_mean_absolute_error: 0.5923
Epoch 43/150

Epoch 00043: val_loss did not improve from 0.92924
84996/84996 - 347s - loss: 0.8903 - mean_absolute_error: 0.3127 - val_loss: 1.1184 - val_mean_absolute_error: 0.4613
Epoch 44/150

Epoch 00044: val_loss did not improve from 0.92924
84996/84996 - 345s - loss: 0.8790 - mean_absolute_error: 0.3087 - val_loss: 1.1082 - val_mean_absolute_error: 0.4587
Epoch 45/150

Epoch 00045: val_loss did not improve from 0.92924
84996/84996 - 346s - loss: 0.8722 - mean_absolute_error: 0.3074 - val_loss: 1.0310 - val_mean_absolute_error: 0.3859
Epoch 46/150

Epoch 00046: val_loss did not improve from 0.92924
84996/84996 - 345s - loss: 0.8638 - me

In [ ]:
print("Evaluate on test data")
results = final_model.evaluate(X_test, y_test, batch_size=32)
print("test loss, test acc:", results)


_history_path = home+ '/repo/planning/traj_pred/total_model_history/veh_history_8kfb.sav'
pickle.dump(history.history, open(_history_path, 'wb'))

_weights_path =  home+ '/repo/planning/traj_pred/total_model_history/veh_weights_8kfb.h5'
final_model.save_weights(_weights_path)


In [ ]:
################################DONE

In [ ]:
# In[ ]:
# X_val, X_test, y_val, y_test,X_train, , y_train, 


In [ ]:
history.history.keys()
ax1,=plt.plot( history.history['loss'], c='b', label='Training loss')
ax2,=plt.plot( history.history['val_loss'], c='r', label='Validation loss')

plt.legend(handles=[ax1,ax2])
plt.title('Loss')
plt.show()

In [ ]:
# final_model.compile(loss=v3_displacement_error, optimizer=adadelta, metrics=mtrc)
# final_model.load_weights('home+ '/repo/planning/traj_pred/total_model_history/veh_weights.h5'')

In [ ]:
# ##prediction
# # from utilities import get_position_sequence
# def get_position_sequence(prev, new):
#     arr = np.delete(prev, 0, 0)
#     a=np.vstack((arr,new))
#     return a 

# def get_sequence_prediction(X_position_test, i):
#     results =[]
#     s=X_position_test[i]
#     for count in range(0,50):
        
#         result = final_model.predict(np.array([s]))
# #         print(result)
#         results.append(result[0])
#         s = get_position_sequence(s, result)
#     results=np.array(results)
# #     print(s.shape,results.shape)
    
#     return results

# # i= np.random.randint(0, len(X_test))
# # results = get_sequence_prediction(X_test, i)
# # print(results.shape)

# test=np.load('vehicle_test_history.npy')
# v_predict=[]
# print(len(test))
# for i in range(len(test)):
#     print(i)
#     results = get_sequence_prediction(test, i)
#     v_predict.append(results)
# np.save("v_predictions",v_predict)

# print('done')

In [ ]:
# print(np.array(v_predict).shape)
# np.save("v_predictions",v_predict)
# print('done')

In [ ]:
# i= np.random.randint(0, len(X_test))


# # In[ ]:
# # i =2273

# # i=1574
# # i=3070

# # i=387,39
# # i=667
# # i=15
# print("showing trip", i)


# # from utilities import get_position_sequence
# def get_position_sequence(prev, new):
#     arr = np.delete(prev, 0, 0)
#     a=np.vstack((arr,new))
#     return a 

# def get_sequence_prediction(X_position_test, i):
#     results =[]
#     s=X_position_test[i]
#     for count in range(0,12):
        
#         result = final_model.predict(np.array([s]))
# #         print(result)
#         results.append(result[0])
#         s = get_position_sequence(s, result)
#     results=np.array(results)
# #     print(s.shape,results.shape)
    
#     return results

# results = get_sequence_prediction(X_test, i)

# # print(len(results))
# # print("sss",results)

# ax1 =plt.scatter(y_test[i][:,0],y_test[i][:,1],color='b',label='Ground truth')

# ax2 =plt.scatter(X_test[i][:,0],X_test[i][:,1],color='r', label='Training segment')


# ax3 =plt.scatter(results[:,0],results[:,1],color='g', label='Prediction')
# # ax3 =plt.scatter(result[0][0],result[0][1],color='g', label='Prediction')

# plt.legend(handles=[ax1, ax2, ax3])
# plt.show()

# distance = v3_displacement_error(results,y_test[i] )
# avg = K.mean(distance)
# pl =[]
# # tf.print(distance,output_stream=sys.stderr)
# with tf.compat.v1.Session() as sess: pl=distance.eval(), print(distance.eval(),'\naverage',avg.eval()) 
# import matplotlib.pyplot as plt
# # print(pl[0])
# plt.plot(pl[0])
# plt.title('Displacement error')
# plt.show()

In [ ]:
# final_displacement=[]
# avg_displacement=[]

# # for idx in range(0,len(X_test)):
# for idx in range(0,1000):
#     p=[]
#     re= get_sequence_prediction(X_test, idx)
#     distance = v3_displacement_error(re,y_test[idx] )
#     avg = K.mean(distance)
    
#     with tf.compat.v1.Session() as sess:
#         p=distance.eval()
#         ax =avg.eval()
# #     print(p,ax)
#     final_displacement.append(p)
#     avg_displacement.append(ax)

# final_displacement = np.array(final_displacement)
# sec=[]

# for j in range(0,final_displacement.shape[1]):
#     displacement = sum(final_displacement[:,j])/len(final_displacement)
    
#     sec.append(displacement)
# print("Final displacement error:",sec[-1])
# print("Average displacement error:",sum(avg_displacement)/len(avg_displacement))

# print(sec)
# plt.plot(sec)
# plt.title("average displacement error at each timestep")
# plt.show()



In [ ]:
# history2 = pickle.load(open( cwd+ '/saved_model/lab1_txt_max_history_mae_adadelta_600.sav',  "rb") ) 
# history3 = pickle.load(open( cwd+ '/saved_model/lab1_txt_max_history_mae_adadelta.sav',  "rb") )
# print(len(history.history['loss']))
# # plt.plot(range(600,1100),history.history['loss'], c='b')
# # plt.plot( range(300,600), history2['loss'], c='b')
# plt.plot( history3['loss'], c='b')
# plt.title('train loss')

# # plt.plot(range(600,1100),history.history['val_loss'], c='r')
# # plt.plot( range(300,600), history2['val_loss'], c='r')
# plt.plot( history3['val_loss'], c='r')
# plt.title('loss')


# plt.show()